In [1]:
import pickle
from forward_model import fwModel 
import sys
sys.path.insert(
    0, '/Users/aibrahi2/Research/Bayesian_Inference-ML-Atmospheric_Correction/src')
import LUT_generator.ac_likelihood_w_chlor as ac_likelihood_w_chlor
from acModel import acModel
import numpyro
from numpyro.infer import MCMC, NUTS
import numpy as onp
from scipy.interpolate import RegularGridInterpolator as RGI
import argparse
import jax.random as random


NN_path = '../../data/NN_model/best_model_pca_tf_lognorm/'
scaler_path = '../../data/NN_model/best_model_pca_tf_lognorm_input_scaler.bin'
fwd = fwModel(NN_path, scaler_path)
fwd.load_NN(scaler_path)
forward_model = fwd.forward_model



Opening /Users/aibrahi2/Research/Bayesian_Inference-ML-Atmospheric_Correction/data/tables/aerosol_ref_transmittance_LUT.nc
Opening /Users/aibrahi2/Research/Bayesian_Inference-ML-Atmospheric_Correction/data/tables/Rayleigh_LUT.nc


/Users/aibrahi2/anaconda3/envs/new_pymc/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.20.3 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded Neural Network model.
Loaded Neural Network model.


In [15]:
def σ_per_geom(solz, relaz, senz):
    diff_per_geom = onp.load('../../data/NN_model/diff_per_geom.npy')
    θ0 = onp.linspace(0, 80, 9)
    ϕ = onp.linspace(0, 180, 19)
    θ = onp.linspace(0, 70, 8)
    return RGI((θ0, ϕ, θ), diff_per_geom)((solz, relaz, senz))

def run_inference(model, rng_key, priors, geom, observations, nir):
    kernel = NUTS(model)
    mcmc = MCMC(kernel, 100, 100, num_chains=1,
                progress_bar=True, chain_method='parallel')
    solz = geom[0]; relaz = geom[1]; senz = geom[2]
    σ_mdl = σ_per_geom(solz, relaz, senz)
    mcmc.run(rng_key, observations, priors, σ_mdl, nir)
    trace = mcmc.get_samples(group_by_chain=True)
    return trace


def main( model, priors, geom, observation, nir):

    global samples

    # do inference
    rng_key = random.PRNGKey(0)
    rng_key, rng_key_ = random.split(rng_key)
    samples = run_inference(model, rng_key_,
                                priors, geom, observation, nir)



In [16]:
assert numpyro.__version__.startswith('0.2.4')
# parser = argparse.ArgumentParser(description="Atmospheric Correction Inference")
# parser.add_argument("-n", "--num_samples", nargs="?",
#                     default=1000, type=int)
# parser.add_argument("--num_warmup", nargs='?', default=1000, type=int)
# parser.add_argument("--num_chains", nargs='?', default=1, type=int)
# parser.add_argument("--device", default='cpu',
#                     type=str, help='use "cpu" or "gpu".')
# args = parser.parse_args()

# numpyro.set_platform(args.device)
# numpyro.set_host_device_count(args.num_chains)


with open('/Users/aibrahi2/Research/atmocor/chlor_based_ret/in_out_na_df.pkl', 'rb') as pickle_file:
    in_out_na_df = pickle.load(pickle_file)

NN = 1
LUT = 0
nir = 0

for i in range(1):
    pr = in_out_na_df['pr'].values[i]
    ws = in_out_na_df['ws'].values[i]
    rh = in_out_na_df['rh'].values[i]
    o3 = in_out_na_df['o3'].values[i]
    fmf = in_out_na_df['fmf'].values[i]
    τ = in_out_na_df['τ'].values[i]
    wv = in_out_na_df['wv'].values[i]
    chlor_a = in_out_na_df['chlor_a'].values[i]
    solz = in_out_na_df['solz'].values[i]
    relaz = in_out_na_df['relaz'].values[i]
    senz = in_out_na_df['senz'].values[i]

    priors = [pr, ws, rh, o3, wv, solz, relaz, senz]
    inputs_ = (([pr, ws, rh, o3, fmf, τ, wv, chlor_a, solz, relaz, senz]))
    if NN:
        ρ_obs = (forward_model(inputs_))[0:13]
    elif LUT:
        ρ_obs = ac_likelihood_w_chlor.get_toa(
            onp.array([pr, ws, rh, o3, fmf, τ, wv, chlor_a]), solz, relaz, senz)[0, 0:13]
geom = (solz, relaz, senz)
print(ρ_obs)
main(acModel, priors, geom, ρ_obs, nir)

[0.3540061  0.31756118 0.28843156 0.26350653 0.2118493  0.19553697
 0.18856955 0.14801267 0.14763978 0.14663665 0.12892458 0.10226407
 0.10208725]


sample: 100%|██████████| 200/200 [00:19<00:00, 10.17it/s, 1023 steps of size 3.83e-03. acc. prob=0.95]


In [18]:
samples.

AttributeError: 'dict' object has no attribute 'summary'

In [16]:
from functools import lru_cache


In [17]:
import functools
import gc

gc.collect()
wrappers = [
    a for a in gc.get_objects() 
    if isinstance(a, functools._lru_cache_wrapper)]

for wrapper in wrappers:
    wrapper.cache_clear()


In [5]:
acModel

<function acModel.acModel(ρ_obs, priors, σ_mdl, nir)>